# Merging CSV files
Code used to merge CSV files

In [ ]:
# Import necessary libraries.
import numpy as np
import pandas as pd
from sklearn import preprocessing
print("Setup Complete")

Setup Complete


In [ ]:
# Import the data.
composition_data = pd.read_csv("/content/ATP3-UFS-Composition.csv", index_col=0, parse_dates=['DATETIME'])
input_data_initial = pd.read_csv("/content/ATP3-UFS-SummaryCombinedData.csv", index_col=0, parse_dates=['DateTime'])
input_data_cleaned = input_data_initial[input_data_initial['Analytical.Sample.ID'].notna()]
temp1 = composition_data.set_index("Analytical.Sample.ID")
temp2 = input_data_cleaned.set_index("Analytical.Sample.ID")
combined_data_init = temp1.merge(temp2)
pd.DataFrame.drop_duplicates(combined_data_init)
combined_data_temp = combined_data_init.drop(['NH4.mg.L', 'NH4.PCT.RSD', 'crash', 'Comments', 'OD680', 'OD680.PCT.RSD',
                                         'NO3.supplied.g', 'NO3.utilized.g', 'NO3.efficiency..', 'NH4.supplied.g',	'NH4.utilized.g',
                                         'NH4.efficiency..', 'P.supplied.g',	'P.utilized.g',	'P.efficiency..', 'mg.AFDW_mol.PAR.input',
                                         'mg.AFDW_kW.GLE.input'], axis=1).rename(columns={'DATETIME': 'Date'})
combined_data_temp['Date'] = pd.to_datetime(combined_data_temp['Date'], format='%d%b%Y')
combined_data_temp['SiteID2'] = combined_data_temp['SiteID']
combined_data_temp = combined_data_temp.set_index(['SiteID', 'Date'])
# combined_data_temp['Date'] = combined_data_temp['Date'].dt.date
# combined_data_temp.Date.astype('str')
# combined_data_temp.set_index('Date')

# combined_data_temp['DateSiteID'] = combined_data_temp['Date']  + combined_data_temp['SiteID']
# combined_data_temp = combined_data_temp.set_index('DateSiteID')
# combined_data_temp.head()
# combined_data_temp.Date.unique()

# date_index = combined_data_temp[['SiteID',	'StrainID', 'Date']]
# combined_data_temp = combined_data_temp.set_index('Date')
# DateTime_unique = input_data_cleaned.DateTime.dt.date
# DateTime_unique.astype('O')
# pd.DataFrame({'Date': DateTime_unique, 'Holder': 1}).set_index('Date')
# DateTime_unique
weather_data = pd.read_csv("/content/ATP3-UFS-Weather-daily.csv", index_col = 0, parse_dates=['Date'])
weather_data = weather_data.set_index(['SiteID', 'Date'])
instrumentation_data = pd.read_csv("/content/ATP3-UFS-Instrumentation-daily.csv", index_col = 0, parse_dates=['Date'])
instrumentation_data = instrumentation_data.set_index(['SiteID', 'Date'])

# use only one of the following two lines
# combined_data = combined_data_temp.merge(weather_data)
combined_data = combined_data_temp.merge(instrumentation_data)

# combined_data.to_csv("/content/drive/My Drive/Colab Notebooks/Combined Data.csv")
combined_data.head

<bound method NDFrame.head of        Protein.AF  FAME.Lipids.AF  ...  Sal (g.L) PAR (umol.m2.s)
0           43.86           10.77  ...      32.11           62.10
1           43.07           13.09  ...      27.13          303.42
2           42.08           14.07  ...      27.13          303.42
3           43.40             NaN  ...      36.09             NaN
4           44.66           13.77  ...      31.51             NaN
...           ...             ...  ...        ...             ...
21769       34.27            7.99  ...       0.83             NaN
21770       34.27            7.99  ...       0.80             NaN
21771       34.27            7.99  ...       0.80             NaN
21772       34.27            7.99  ...       0.50             NaN
21773       34.27            7.99  ...       0.40             NaN

[21774 rows x 46 columns]>